# 第五章 基于文档的问答

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [一、导入embedding模型和向量存储组件](#一、导入embedding模型和向量存储组件)
     - [1.1 创建向量存储](#1.1-创建向量存储)
     - [1.2 使用语言模型与文档结合使用](#1.2-使用语言模型与文档结合使用)
 - [二、 如何回答我们文档的相关问题](#二、-如何回答我们文档的相关问题)


常见的应用场景是使用大语言模型来构建一个能够回答关于<span style="color:red">给定文档和文档集合</span>的问答系统。对于给定的文档, 比如从PDF、网页、公司主页中提取构建的内部文档集合，我们可以使用大语言模型来回答关于这些文档内容的问题，以帮助用户更有效地获取和使用他们所需要的信息。这种方式非常有效且灵活地适用于实际应用场景，因为它不仅仅利用大语言模型已有的训练集数据信息，它还能使用<span style="color:red">外部信息</span>。

这个过程，我们会涉及LongChain中的其他组件，比如：`表征模型（Embedding Models)`和`向量储存(Vector Stores)`

## 一、设置OpenAI API Key


登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

## 二、 结合表征模型和向量存储

使用语言模型与文档结合使用时，语言模型一次只能使用几千个单词的信息。如果我们文档比较长，如何让语言模型回答关于其中所有内容的问题呢？我们通过几个通过向量表征（Embeddings）和向量存储(Vector Store)实现

`文本表征（Embeddings)`是对文本语义的向量表征，相似内容的文本具有相似的表征向量。这使我们可以<span style="color:red">在向量空间中比较文本的相似性</span> 。

`向量数据库(Vector Database)`用来存储文档的文本块。对于给定的文档，我们首先将其分成较小的`文本块（chunks）`，然后获取每个小文本块的文本表征，并将这些表征储存在向量数据库中。这个流程正是创建索引(index)的过程。将文档分成小文本块的原因在于我们可能无法将整个文档传入语言模型进行处理。

索引创建完成后，我们可以使用索引来<span style="color:red">查找与传入的查询(Query)最相关的文本片段</span> - 首先我们为查询获取向量表征，然后我们将其与向量数据库中的所有向量进行比较并选择最相似的n个文本块，最后我们将这些相似的文本块构建提示，并输入到语言模型，从而得到最终答案。

### 2.1 导入数据

In [ ]:
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储

#创建一个文档加载器，通过csv格式加载
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

In [8]:
#查看单个文档，每个文档对应于CSV中的一行数据
docs[2]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 2}, page_content=": 2\nname: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece\ndescription: She'll love the bright colors, ruffles and exclusive whimsical prints of this toddler's two-piece swimsuit! Our four-way-stretch and chlorine-resistant fabric keeps its shape and resists snags. The UPF 50+ rated fabric provides the highest rated sun protection possible, blocking 98% of the sun's harmful rays. The crossover no-slip straps and fully lined bottom ensure a secure fit and maximum coverage. Machine wash and line dry for best results. Imported.")

### 2.2 文本向量表征模型

<span style="color:#00ff00">
需要使用专门的<code>嵌入模型（embedding models)</code>将文本嵌入为向量表示。
</span>

In [9]:
#使用OpenAIEmbedding类
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small') #初始化

In [10]:
#因为文档比较短了，所以这里不需要进行任何分块,可以直接进行向量表征
#使用初始化OpenAIEmbedding实例上的查询方法embed_query为文本创建向量表征
embed = embeddings.embed_query("Hi my name is Harrison")

#查看得到向量表征的长度
print(len(embed))

1536


In [11]:
#每个元素都是不同的数字值，组合起来就是文本的向量表征
print(embed[:5])

[0.005708176642656326, -0.005911823362112045, -0.06263796985149384, 0.02263817936182022, -0.051403988152742386]


### 2.3 基于向量表征创建向量存储

<span style="color:#00ff00">
利用LangChain中的<code>DocArrayInMemorySearch</code>类创建向量存储。
</span>

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch #向量存储

# 将刚才创建文本向量表征(embeddings)存储在向量存储(vector store)中
# 使用DocArrayInMemorySearch类的from_documents方法来实现
# 该方法接受文档列表以及向量表征模型作为输入
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

### 2.4 查询创建的向量存储

In [16]:
# 由于文档是英文，我们采用英文的询问

query = "Please suggest a shirt with sunblocking"

#使用上面的向量存储来查找与传入查询类似的文本，得到一个相似文档列表
docs = db.similarity_search(query)

In [17]:
#打印返回文档的个数
len(docs)

4

In [18]:
#打印返回的第一个文档
docs[0] 

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

我们可以看到一共返回了四个结果。输出的第一结果是一件关于防晒的衬衫，满足我们查询的要求：`请推荐一件防晒功能的衬衫（Please suggest a shirt with sunblocking）`

### 2.5 使用向量储存回答文档的相关问题

In [21]:
from langchain.chat_models import init_chat_model

#导入大语言模型
llm = init_chat_model(model="gpt-4o-mini",temperature = 0.0)

In [22]:
#合并获得的相似文档内容
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

<span style="color:#00ff00">
LangChain的新版api中使用<code>invoke</code>方法调用llm。
</span>

In [37]:
from langchain_core.messages import HumanMessage

#将合并的相似文档内容后加上问题（question）输入到 `llm.call_as_llm`中
#这里问题是：以Markdown表格的方式列出所有具有防晒功能的衬衫并总结
response = llm.invoke([HumanMessage(content=f"{qdocs} Question: Please list all your shirts with sun protection in a table \
in markdown and summarize each one.")])

<span style="color:#00ff00">
这里返回的结果是一个<code>AIMessage</code>对象，需要使用<code>content</code>属性来获取结果。
</span>

In [39]:
response


AIMessage(content="Here's a table summarizing the shirts with sun protection:\n\n| Name                                   | Description Summary                                                                                                                                                                                                                     |\n|----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n| **Sun Shield Shirt**                  | High-performance sun shirt with UPF 50+ protection, blocking 98% of UV rays. Slightly fitted, made of 78% nylon and 22% Lycra Xtra Life fiber. Moisture-wicking, abrasion-resistant, and fits over swimsuits. Handwash recommended.         |\n| **Men's Tropical Plaid Short-Sleeve Shirt** | Lightest hot-weather shirt rated

In [38]:
from IPython.display import display, Markdown #在jupyter显示信息的工具

display(Markdown(response.content))

Here's a table summarizing the shirts with sun protection:

| Name                                   | Description Summary                                                                                                                                                                                                                     |
|----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Sun Shield Shirt**                  | High-performance sun shirt with UPF 50+ protection, blocking 98% of UV rays. Slightly fitted, made of 78% nylon and 22% Lycra Xtra Life fiber. Moisture-wicking, abrasion-resistant, and fits over swimsuits. Handwash recommended.         |
| **Men's Tropical Plaid Short-Sleeve Shirt** | Lightest hot-weather shirt rated UPF 50+ for sun protection. Traditional fit, made of 100% wrinkle-resistant polyester. Features front and back cape venting and two front bellows pockets. Provides high sun protection.                  |
| **Men's Plaid Tropic Shirt, Short-Sleeve** | Ultracomfortable shirt with UPF 50+ protection, originally designed for fishing. Made of 52% polyester and 48% nylon, wrinkle-free, and moisture-evaporating. Features cape venting and two front bellows pockets. Machine washable.         |
| **Tropical Breeze Shirt**              | Lightweight, breathable long-sleeve shirt with UPF 50+ protection. Traditional fit, made of 71% nylon and 29% polyester. Features moisture-wicking, wrinkle-resistant fabric, cape venting, and two front bellows pockets. Machine washable. |

This table provides a concise overview of each shirt's features and benefits related to sun protection.

在此处打印响应，我们可以看到我们得到了一个表格，正如我们所要求的那样

### 2.7 通过LangChain链进行封装

我们创建一个<code>检索QA链</code>，用于对检索到的文档进行问题回答。要创建这样的链，我们将传入几个不同的东西
- 1、语言模型，在最后进行文本生成
- 2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
- 3、传入一个检索器



在 LangChain 中，<code>chain_type="stuff"</code> 参数表示使用 "Stuff" 文档处理策略，这是处理检索到的文档的一种方式。"Stuff" 策略的特点：
- 1.将所有检索到的文档内容简单地合并成一个大文档
- 2.将这个合并后的文档与问题一起一次性传递给语言模型
- 3.让语言模型基于所有文档内容生成一个答案
- 4.只调用语言模型一次，因此速度较快

这是最简单的方法，但有<span style="color:red">限制</span>：只适用于检索到的文档总量较小的情况，并且受语言模型最大输入长度的限制。如果文档内容超过模型的上下文窗口大小，就不能使用此方法。

笔记本最后部分提到了其他几种策略：
- 1.<code>Map Reduce</code>：分别处理每个文档后再合并结果
- 2.<code>Refine</code>：逐步处理文档并细化答案
- 3.<code>Map Re-rank</code>：对每个文档生成答案并排序

In [45]:
from langchain.chains.retrieval_qa.base import RetrievalQA #检索QA链，在文档上进行检索

#基于向量储存，创建检索器
retriever = db.as_retriever()

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [30]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."#创建一个查询并在此查询上运行链

<span style="color:#00ff00">
同样，应该使用<code>invoke</code>方法调用查询链。
</span>

In [46]:
response = qa_stuff.invoke({"query": query})



> Entering new RetrievalQA chain...

> Finished chain.


<span style="color:#00ff00">
这里返回的结果是一个<code>字典</code>对象，需要使用<code>result</code>键来获取结果。
</span>

In [47]:
response

{'query': 'Please list all your shirts with sun protection in a table in markdown and summarize each one.',
 'result': "Here is a table summarizing the shirts with sun protection:\n\n| Name                                   | Description                                                                                                           |\n|----------------------------------------|-----------------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt                       | High-performance sun shirt with UPF 50+ protection, made of 78% nylon and 22% Lycra Xtra Life. Moisture-wicking, abrasion-resistant, and fits comfortably over swimsuits. Handwash recommended. |\n| Men's Tropical Plaid Short-Sleeve Shirt| Lightest hot-weather shirt with UPF 50+ protection, made of 100% polyester. Features a relaxed fit, cape venting, and two front bellows pockets. Wrinkle-resistant. |\n| Men's Plaid Tropic Shirt, Short-Sleeve 

In [48]:
display(Markdown(response["result"] ))

Here is a table summarizing the shirts with sun protection:

| Name                                   | Description                                                                                                           |
|----------------------------------------|-----------------------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt                       | High-performance sun shirt with UPF 50+ protection, made of 78% nylon and 22% Lycra Xtra Life. Moisture-wicking, abrasion-resistant, and fits comfortably over swimsuits. Handwash recommended. |
| Men's Tropical Plaid Short-Sleeve Shirt| Lightest hot-weather shirt with UPF 50+ protection, made of 100% polyester. Features a relaxed fit, cape venting, and two front bellows pockets. Wrinkle-resistant. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable shirt with UPF 50+ protection, made of 52% polyester and 48% nylon. Designed for fishing, it features cape venting, two front bellows pockets, and is machine washable. |
| Tropical Breeze Shirt                  | Lightweight, breathable long-sleeve shirt with UPF 50+ protection, made of 71% nylon and 29% polyester. Features moisture-wicking, wrinkle-resistant fabric, cape venting, and is machine washable. |

Each shirt offers high-level sun protection, making them suitable for outdoor activities.

这两个方式返回相同的结果。

### 2.8 不同的文档处理策略

想在许多不同类型的块上执行相同类型的问答，该怎么办？之前的实验中只返回了4个文档，如果有多个文档，那么我们可以使用几种不同的方法
* `Map Reduce`  
将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行。可以并行处理单个问题，同时也需要更多的调用。它将所有文档视为独立的
* `Refine`    
用于循环许多文档，实际上它是用迭代实现的，它建立在先前文档的答案之上，非常适合用于合并信息并随时间逐步构建答案，由于依赖于先前调用的结果，因此它通常需要更长的时间，并且基本上需要与Map Reduce一样多的调用
* `Map Re-rank`   
对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分，并在那里精细调整说明，可以批量处理它们相对较快，但是更加昂贵
* `Stuff`    
将所有内容组合成一个文档